In [ ]:
!pip install torch
!pip install torch-geometric

This notebook is to visualize the generation of the initial embedding and target embedding from lightGCN.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.ERROR)

import numpy as np
import pandas as pd
import sys
import os

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Import modules from the utils folder
from utils.dataloader import DataLoader
from utils.data_split import train_test_split

In [4]:
movie_data = DataLoader(size="100k")
data = movie_data.load_ratings()
train_set, test_set = train_test_split(data)

In [5]:
training_ratings = train_set

In [6]:
len(training_ratings['item'].unique())

1609

In [7]:
# Calculate user degree
user_degree_df = training_ratings.groupby("user").size().reset_index(name="degree")

# Calculate item degree
item_degree_df = training_ratings.groupby("item").size().reset_index(name="degree")

# Determine the cutoff for top 50% users and items based on degree
user_cutoff = int(len(user_degree_df) * 0.5)
item_cutoff = int(len(item_degree_df) * 0.5)

# Get top 50% users and items
top_users_df = user_degree_df.nlargest(user_cutoff, 'degree')[['user']]
top_items_df = item_degree_df.nlargest(item_cutoff, 'degree')[['item']]

# Display the DataFrames
print("Top 50% Users DataFrame:")
print(top_users_df.shape)

print("\nTop 50% Items DataFrame:")
print(top_items_df.shape)

Top 50% Users DataFrame:
(471, 1)

Top 50% Items DataFrame:
(804, 1)


In [8]:
filtered_user_movie_rating = training_ratings[
    (training_ratings['user'].isin(top_users_df['user'])) &
    (training_ratings['item'].isin(top_items_df['item']))
]
filtered_user_movie_rating

,user,item,rating,timestamp
85,332,566,4,888360342
86,332,451,5,888360179
87,332,595,4,887938574
88,332,44,3,888360342
90,332,258,5,887916151
...,...,...,...,...
65546,436,427,3,887769105
65547,436,234,3,887769471
65548,436,187,5,887768982
65551,436,856,4,887769952


In [9]:
import torch
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from lightgcn_model import LightGCN, LightGCNModel

filtered_user_movie_rating = filtered_user_movie_rating[['user','item','rating']]

# Assuming filtered_user_movie_rating is already available
# and contains the columns 'user', 'item', and 'rating'.

# Split the filtered dataset into training and testing sets
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(filtered_user_movie_rating, test_size=0.3, random_state=42)

# Instantiate LightGCNModel with the filtered data
model = LightGCNModel(
    train_set=train_set,
    test_set=test_set,
    num_layers=3,        
    embedding_dim=64,    
    learning_rate=0.01,  
    epochs=50,          
    device='cpu'         
)

# Prepare training data (includes building graph and initializing model)
model.prepare_training_data()


with torch.no_grad():  
    user_embeddings, item_embeddings = model.model(model.adj_norm)

print("User Embeddings Shape:", user_embeddings.shape)  # Shape: (num_users, embedding_dim)
print("Item Embeddings Shape:", item_embeddings.shape)  # Shape: (num_items, embedding_dim)


User Embeddings Shape: torch.Size([471, 64])
Item Embeddings Shape: torch.Size([804, 64])


In [10]:
# Convert user_embeddings to DataFrame
target_user_embeddings = pd.DataFrame({
    'user': [model.idx2user[i] for i in range(user_embeddings.shape[0])],  # Map row indices to user IDs
    'embedding': user_embeddings.cpu().numpy().tolist()  # Convert tensor rows to lists
})

# Convert item_embeddings to DataFrame
target_item_embeddings = pd.DataFrame({
    'item': [model.idx2item[i] for i in range(item_embeddings.shape[0])],  # Map row indices to item IDs
    'embedding': item_embeddings.cpu().numpy().tolist()  # Convert tensor rows to lists
})

# Display the DataFrames
print("User Embeddings DataFrame:")
print(target_user_embeddings.shape)

print("\nItem Embeddings DataFrame:")
print(target_item_embeddings.shape)


User Embeddings DataFrame:
(471, 2)

Item Embeddings DataFrame:
(804, 2)


In [11]:
import torch.nn as nn

# Step 1: Define the number of users and embedding dimension
num_users = len(top_users_df['user'])  # Assuming user_degree_df contains the user IDs
num_items = len(top_items_df['item'])
embedding_dim = 64  # Specify embedding dimension

# Step 2: Initialize the embedding layer
init_user_embedding = nn.Embedding(num_users, embedding_dim)
init_item_embedding = nn.Embedding(num_items, embedding_dim)

# Step 3: Apply Xavier Uniform Initialization
nn.init.xavier_uniform_(init_user_embedding.weight)
nn.init.xavier_uniform_(init_item_embedding.weight)

# Step 4: Convert initialized embeddings into a DataFrame
user_ids = top_users_df['user'].values  # Extract user IDs from top_users_df
user_embeddings = init_user_embedding.weight.cpu().detach().numpy()  # Convert to numpy array
item_ids = top_items_df['item'].values  # Extract item IDs from top_users_df
item_embeddings = init_item_embedding.weight.cpu().detach().numpy()  # Convert to numpy array

# Create DataFrame
user_embeddings_df = pd.DataFrame({
    'user': user_ids,  # Map rows to user IDs
    'embedding': user_embeddings.tolist()  # Convert each embedding row to a list
})

item_embeddings_df = pd.DataFrame({
    'item': item_ids,  # Map rows to item IDs
    'embedding': item_embeddings.tolist()  # Convert each embedding row to a list
})

# Display the resulting DataFrame
print("User Embeddings DataFrame:")
print(user_embeddings_df.shape)

print("Item Embeddings DataFrame:")
print(item_embeddings_df.shape)


User Embeddings DataFrame:
(471, 2)
Item Embeddings DataFrame:
(804, 2)


In [12]:
# Save to CSV

# Define the folder name
output_folder = "ml_gnn_ebd"

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save the CSV files into the specified folder
user_embeddings_df.to_csv(os.path.join(output_folder, "initial_user_ebds.csv"), index=False)
item_embeddings_df.to_csv(os.path.join(output_folder, "initial_item_ebds.csv"), index=False)
target_user_embeddings.to_csv(os.path.join(output_folder, "target_user_ebds.csv"), index=False)
target_item_embeddings.to_csv(os.path.join(output_folder, "target_item_ebds.csv"), index=False)